<a href="https://colab.research.google.com/github/leekh8/Colaboratory/blob/main/OpenCV_004_FastAPI_%EB%B0%B0%ED%8F%AC%EC%9A%A9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img src="https://arome1004.cafe24.com/images/opencv/lecture_image/00_title.png">

<table border=1 width=100%>
    <tr><td style="border: 1px solid black; width:600px; height:40px; text-align: center;"><font size=4 color=blue><b>[4차시] 학습목표</b></font></td></tr>       
    <tr><td style="border: 1px solid black; text-align: left;"><font size=3>
        
○ 기본적인 FastAPI 사용법 학습하기<br>

○ 웹 화면에 카메라 영상 출력하기<br>

○ 웹 출력 영상을 Gray 이미지로 출력하기        
</font></td></tr>   
</table>

# FastAPI 설치 및 서버 실행하기

- <font color=red>FastAPI</font> : Sebastián Ramírez란 사람이 만든 파이썬 기반 오픈소스 웹 프레임워크
  - 파이썬 3.6 이상
  - 빠른(고성능) API 서버 웹 프레임워크
  - Flask나 Django보다 단순하고 배우기 쉬움
  - GitHub : https://github.com/tiangolo/fastapi
  - 튜토리얼 : https://fastapi.tiangolo.com/ko/
  - 다양한 형태의 Response : https://fastapi.tiangolo.com/ko/advanced/custom-response/


- ASGI(Asynchronous Server Gateway Interface) : 비동기 Python 웹 서버  
  - 동기(synchronous) 처리 : 특정 작업이 끝나면 다음 작업을 처리하는 순차처리 방식
    - 파이썬은 기본적으로 동기 방식으로 동작
  - 비동기(asynchronous) 처리 : 여러 작업을 처리하도록 예약한 뒤 작업이 끝나면 결과를 받는 방식
    - 스레드 하나로 동시 처리를 하기 위한 것

  <img src="https://arome1004.cafe24.com/images/opencv/lecture_image/04_fastapi06.png" width=50%>  
  

- <font color=red>pyngrok</font> : 방화벽을 넘어서 외부에서 로컬에 접속 가능하게 하는 터널 프로그램


- <font color=red>asyncio</font> : Python 표준 라이브러리와 함께 제공되는 비동기 I/O 프레임워크로 파이썬 3.4에서 추가


- <font color=red>async / await</font> : 비동기(Asynchronous) 처리를 위해 파이썬 3.5에서 추가한 구문
  - async : 비동기 함수를 생성하는 명령
  - await : 비동기 함수를 호출하는 명령
  
  
- <font color=red>nest_asyncio</font> : 비동기 방식에서 이벤트가 중첩되는 것을 허용하는 기능을 제공하는 라이브러리
  
  
- <font color=red>uvloop</font> : asyncio를 대체하기 위해 제작
  - 동기 함수 내에서 비동기 함수를 호출하려면 asyncio 라이브러리의 이벤트 루프를 이용하는 데 이를 대신하는 기능
  
  - Cython으로 작성
  - libuv 위에 구축 (libuv : nodejs에서 사용하는 고성능 다중 플랫폼 비동기 I / O 라이브러리)
  - 성능상에서 nodejs, gevent 및 기타 Python 비동기 프레임 워크보다 2배 이상 빠름
    
  
- <font color=red>uvicorn</font> : uvloop 및 httptools를 사용하여 만든 빠른 ASGI (Asynchronous Server Gateway Inteface) 서버
  - 다양한 관리 기능으로 fastapi와 함께 사용
  - HTTP / 1.1 및 WebSocket을 지원

In [22]:
from google.colab import drive
drive.mount('/content/drive')
%cd '/content/drive/MyDrive/Colab Notebooks/Colaboratory/AISchool/'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/Colab Notebooks/Colaboratory/AISchool


## 동기 프로그래밍

In [ ]:
#실습
import time

def find_users_sync(n):
  for i in range(1, n+1):
    print(f"{n}명 중 {i} 번째 사용자 조회 중")
    time.sleep(1)

  print(f"총 {n}명의 사용자 조회 완료")

find_users_sync(5)

5명 중 1 번째 사용자 조회 중
5명 중 2 번째 사용자 조회 중
5명 중 3 번째 사용자 조회 중
5명 중 4 번째 사용자 조회 중
5명 중 5 번째 사용자 조회 중
총 5명의 사용자 조회 완료


- 3개의 요청을 동기 처리

In [ ]:
#실습
def process_sync():
  start = time.time()

  find_users_sync(3)
  find_users_sync(2)
  find_users_sync(1)

  end = time.time()

  print(f"동기 처리 시 소요시간: {end - start}")

process_sync()

3명 중 1 번째 사용자 조회 중
3명 중 2 번째 사용자 조회 중
3명 중 3 번째 사용자 조회 중
총 3명의 사용자 조회 완료
2명 중 1 번째 사용자 조회 중
2명 중 2 번째 사용자 조회 중
총 2명의 사용자 조회 완료
1명 중 1 번째 사용자 조회 중
총 1명의 사용자 조회 완료
동기 처리 시 소요시간: 6.007686376571655


## 비동기 프로그래밍

In [ ]:
#실습
!pip install nest_asyncio

In [ ]:
#실습
import time
import asyncio

async def find_users_async(n):
  for i in range(1, n+1):
    print(f"{n}명 중 {i} 번째 사용자 조회 중")
    await asyncio.sleep(1)

  print(f"총 {n}명의 사용자 조회 완료")

find_users_async(5)

<coroutine object find_users_async at 0x7b9caec5f920>

# 비동기 함수 호출하기
- ## await: 1개의 비동기 함수 호출
- ## await asyncio.wait(): 여러개의 비동기 함수 호출
- ## asyncio.run(): 비동기 함수 실행

In [ ]:
import nest_asyncio

# 작업 간 메시지 교환이 되도록 설정
nest_asyncio.apply()

async def process_async():
  start = time.time()

  # 여러개의 비동기 함수 호출
  await asyncio.wait(
      [find_users_async(3),
       find_users_async(2),
       find_users_async(1)])

  end = time.time()

  print(f"비동기 처리 시 소요시간: {end - start}")

asyncio.run(process_async())

<ipython-input-9-f7963a81dfa4>:10: DeprecationWarning: The explicit passing of coroutine objects to asyncio.wait() is deprecated since Python 3.8, and scheduled for removal in Python 3.11.
  await asyncio.wait(


1명 중 1 번째 사용자 조회 중
3명 중 1 번째 사용자 조회 중
2명 중 1 번째 사용자 조회 중
총 1명의 사용자 조회 완료
3명 중 2 번째 사용자 조회 중
2명 중 2 번째 사용자 조회 중
3명 중 3 번째 사용자 조회 중
총 2명의 사용자 조회 완료
총 3명의 사용자 조회 완료
비동기 처리 시 소요시간: 3.005168914794922


##  FastAPI 설치하기

In [ ]:
!pip install fastapi uvicorn pyngrok nest_asyncio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.2/66.2 kB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.5/59.5 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 698.7/698.7 kB 5.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.1 MB/s eta 0:00:00
  Created wheel for pyngrok: filename=pyngrok-6.1.0-py3-none-any.whl size=20583 sha256=8f4fec1c4faeaddc95e3b2802e17e6fee6d3953943a2977b908c750b4077902d
  Stored in directory: /root/.cache/pip/wheels/d8/2d/7a/97a039fca211fa789bffad50ff97dca13c01e9b83e8879f503
Successfully built pyngrok


## FastAPI 서버 실행하기

- 참고 : https://blog.naver.com/PostView.naver?blogId=duqrlwjddns1&logNo=222268635237&parentCategoryNo=&categoryNo=&viewDate=&isShowPopularPosts=false&from=postView


- 비동기 처리(asyncio)는 Jupyter Notebook 환경에서 지원하지 않음


- <font color=red>app = FastAPI()</font> : FastAPI 객체를 app 변수에 할당
- <font color=red>nest_asyncio.apply()</font> : 이벤트 루프
- <font color=red>@app.get("/")</font> : GET 메소드로 root url을 접속하는 경우
- <font color=red>async def root()</font> : root() 비동기 함수 (native coroutine)을 만듬
- <font color=red>return {"message": "Hello World"}</font> : message 변수로 해당 값을 JSON 형태로 반환
- <font color=red>uvicorn.run(app, host="127.0.0.1", port=8000)</font> : app 객체에 IP와 Port 설정

<img src="https://arome1004.cafe24.com/images/opencv/lecture_image/04_fastapi01.png" width=50%>

In [ ]:
#실습
from fastapi import FastAPI
import nest_asyncio
import uvicorn
from google.colab.output import eval_js

# colab의 proxy 주소 출력
print(eval_js("google.colab.kernel.proxyPort(5000)"))

# FastAPI 초기화
app = FastAPI()

# 메시지 루프 시작
nest_asyncio.apply()

# 웹 서버 설정
@app.get("/")
async def root():
  return {"message" : "Hello~ World~"}

# 해당 코드를 단독으로 구동하려고 할 때 사용
if __name__ == "__main__":
  uvicorn.run(app, host = "127.0.0.1", port = 5000)

https://pi4kw8px99s-496ff2e9c6d22116-5000-colab.googleusercontent.com/


INFO:     Started server process [195]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:5000 (Press CTRL+C to quit)


INFO:     127.0.0.1:45476 - "GET / HTTP/1.1" 200 OK
INFO:     127.0.0.1:45486 - "GET /favicon.ico HTTP/1.1" 404 Not Found


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [195]


### URL를 함수로 연결하기

- <font color=red>@app.get("/hello")</font>
  - URL에  함수명을 설정
  - 브라우저에서 IP와 포트로 서버 접속하고 URL로 함수명을 넘김
  
  <img src="https://arome1004.cafe24.com/images/opencv/lecture_image/04_fastapi02.png" width=50%>

In [21]:
#실습
from fastapi import FastAPI
import nest_asyncio
import uvicorn
from google.colab.output import eval_js

# colab의 proxy 주소 출력
print(eval_js("google.colab.kernel.proxyPort(5000)"))

# FastAPI 초기화
app = FastAPI()

# 메시지 루프 시작
nest_asyncio.apply()

# 웹 서버 설정
@app.get("/")
async def root():
  return {"message" : "Hello~ World~"}

@app.get("/hello")
async def hello():
  return {"message" : "방가방갑습니다"}

@app.get("/hello/{val}")
async def hello(val):
  return {"message" : "방가방갑습니다 => " + val}

# 해당 코드를 단독으로 구동하려고 할 때 사용
if __name__ == "__main__":
  uvicorn.run(app, host = "127.0.0.1", port = 5000)

https://u0h6fgipdp-496ff2e9c6d22116-5000-colab.googleusercontent.com/


INFO:     Started server process [195]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:5000 (Press CTRL+C to quit)


INFO:     127.0.0.1:43796 - "GET / HTTP/1.1" 200 OK
INFO:     127.0.0.1:43806 - "GET /favicon.ico HTTP/1.1" 404 Not Found
INFO:     127.0.0.1:43808 - "GET /hello/500 HTTP/1.1" 200 OK
INFO:     127.0.0.1:43820 - "GET /favicon.ico HTTP/1.1" 404 Not Found


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [195]


In [23]:
%%writefile ./templates/hello.html
<html><body>
<fint size=10 color=red>Hello World !!</font>
</body></html>

Writing ./templates/hello.html


In [24]:
from fastapi import FastAPI
import nest_asyncio
import uvicorn
from google.colab.output import eval_js
from fastapi.responses import FileResponse

# colab proxy 주소 출력
print(eval_js("google.colab.kernel.proxyPort(5000)"))

#FastAPI 초기화
app=FastAPI()

#메시지 루프시작
nest_asyncio.apply()

# 웹 서버 설정
@app.get("/", response_class=FileResponse)
async def root() :
  return "./templates/hello.html"

# 해당 코드를 단독으로 구동하고자 할 때 사용
if __name__ == "__main__" :
  uvicorn.run(app, host="127.0.0.1", port=5000)

https://u0h6fgipdp-496ff2e9c6d22116-5000-colab.googleusercontent.com/


INFO:     Started server process [195]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:5000 (Press CTRL+C to quit)


INFO:     127.0.0.1:38120 - "GET / HTTP/1.1" 200 OK
INFO:     127.0.0.1:38130 - "GET /favicon.ico HTTP/1.1" 404 Not Found


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [195]


### URL을 변수로 사용하기

- <font color=red>@app.get("/hello/{value}")</font>
  - URL에  value 값을 설정
  - 브라우저에서 IP와 포트로 서버 접속하고 URL로 100을 넘김
    
  <img src="https://arome1004.cafe24.com/images/opencv/lecture_image/04_fastapi03.png" width=50%>     

In [ ]:
#실습

### html 문서를 반환하기

#### 코드 형태로 반환하기

- <font color=red>@app.get("/", response_class=HTMLResponse)</font>
  - HTML 문서를 반환

- 브라우저에서 IP와 포트로 서버 접속

  <img src="https://arome1004.cafe24.com/images/opencv/lecture_image/04_fastapi04.png" width=50%>  

In [ ]:
#실습

#### 파일 형태로 반환하기

- <font color=red>@app.get("/", response_class=FileResponse)</font>
  - 파일 문서를 반환

In [ ]:
#실습

In [ ]:
#실습

### 이미지가 포함된 HTML 문서 반환하기

- static 폴더 : 자원을 담아 놓는 폴더
- static 폴더에 이미지 폴더를 만들고 출력할 이미지를 저장

In [ ]:
!pip install jinja2

In [ ]:
#실습

- <font color=red>Jinja2Templates(directory="templates/")</font>

  - Jinja2 템플릿 사용
  
  
- <font color=red>app.mount("/static", StaticFiles(directory="static", html=True), name="static")</font>  

  - "/static" : 마운트할 경로
  - directory="static" : 정적파일이 들어 있는 폴더명
  - html=True : HTML 문서 여부
  - name="static" : FastAPI에서 내부적으로 사용할 이름
  
  
- <font color=red>templates.TemplateResponse("imgdisp2.html", {"request": request})  </font>  

  - Jinja 템플릿으로 구성된 HTML 문서를 request한 위치로 반환

In [ ]:
#실습

# 데이터 스트리밍 구현

## 구현 방법

- <font color=red>StreamingResponse(test())</font>
  - 스트리밍 함수로 test()를 실행
  
  <img src="https://arome1004.cafe24.com/images/opencv/lecture_image/04_fastapi05.png" width=50%>  

In [ ]:
#실습

- yield() 함수 배우기

  
  <img src="https://arome1004.cafe24.com/images/opencv/lecture_image/04_yield.png" width=70%>  

In [ ]:
#실습

In [ ]:
#실습

## 웹 브라우저에 카메라 영상 출력하기

- index.html를 작성하고 templates 폴더에 저장
  - video_feed : 실행할 함수명
  
  
- Jinja 템플릿
    - 간단한 문법으로 html에서 {}, {{}} 등의 규칙을 이용하여 python 프로그래밍이 가능
    - 데이터를 담은 변수를 return값으로 html과 함께 넘겨준 뒤, html에서 jinja 템플릿 규칙에 따라 화면에 표현할 수 있음
    - Jinja 템플릿 html에서 주석은 {# 주석으로 처리할 내용 #}으로 표시
    - Jinja2 Tempaltes 변수 넘기기 : {{ 변수명 }}
    
  - 참고
    - https://velog.io/@jewon119/01.Flask-기초-Jinja-template
    - https://jinja.palletsprojects.com/en/2.11.x/templates/
  
  
- <font color=red>url_for()</font> : URL 주소값을 가져온다

In [ ]:
#실습

- <font color=red>ret, buffer = cv2.imencode('.jpg', frame)</font> : frame 이미지를 jpg로 인코딩

- <font color=red>frame = buffer.tobytes()</font> : 전송을 위해 인코딩된 이미지를 byte 형식으로 변환


- <font color=red>yield (b'--frame\r\n' b'Content-Type: image/jpeg\r\n\r\n' + frame + b'\r\n') </font>

  - b : byte 형식임을 의미
  - --frame : 프레임을 표시
  - Content-Type: image/jpeg : 문서가 jpg 이미지임을 표시
  
  
- <font color=red>StreamingResponse(get_frames(), media_type="multipart/x-mixed-replace; boundary=frame")</font>

  - get_frames() : 호출할 함수명
  - mimetype : 클라이언트에게 전송된 문서의 타입을 알려주기 위한 파라미터 (type/subtype)
  - multipart : 복합문서 타입 (파일, 영상 등)을 의미
  - x-mixed-replace : x (추가적인 확장 형식), mixed (복합문서), repalce (subtype을 다음 메시지로 대체)
  - boundary : 복합문서 내의 각 문서들을 구분하는 분리자 (동영상이므로 frame으로 구분)

- 이미지 전송 부분을 동기방식으로 구현하는 경우

In [ ]:
#실습

- 이미지 전송 부분을 비동기 방식으로 구현한 경우

In [25]:
%%writefile ./templates/index.html
<html><body><h1>VIDEO STREAMING</h1><img src = "{{ url_for('video_feed') }}" width = "50%"><br /></body></html>

Overwriting ./templates/index.html


In [27]:
# 실습
import cv2

cap = cv2.VideoCapture("./images/KakaoTalk_20230914_113812278.mp4")

async def get_frames():

  while True:
    ret, frame = cap.read()

    if not ret:
      continue

    else:
      ret, buffer = cv2.imencode('.jpg', frame)
      frame = buffer.tobytes()

      yield (b'--frame\r\n' b'Content-Type: image/jpeg\r\n\r\n' + frame + b'\r\n')

  cap.release()

In [32]:
from fastapi import FastAPI, Request
import nest_asyncio
import uvicorn
from google.colab.output import eval_js
from fastapi.responses import StreamingResponse
from fastapi.templating import Jinja2Templates

from google.colab.output import eval_js

print(eval_js("google.colab.kernel.proxyPort(5000)"))

app = FastAPI()
nest_asyncio.apply()
templates = Jinja2Templates(directory = "templates")

@app.get("/")
async def index(request:Request):
  return templates.TemplateResponse("index.html", {"request": request})

@app.get("/video_feed")
async def video_feed():
  return StreamingResponse(await(get_frames()), mimetype = 'multipart/m-mixed-replace; boundary = frame');

if __name__ == "__main__":
  uvicorn.run(app,host="127.0.0.1", port=5000)

https://u0h6fgipdp-496ff2e9c6d22116-5000-colab.googleusercontent.com/


INFO:     Started server process [195]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:5000 (Press CTRL+C to quit)


INFO:     127.0.0.1:45232 - "GET / HTTP/1.1" 200 OK
INFO:     127.0.0.1:45246 - "GET /favicon.ico HTTP/1.1" 404 Not Found


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [195]


## 웹 출력 영상을 Gray 이미지로 출력하기

In [ ]:
#실습

In [ ]:
#실습

<table border=1 width=100%>
    <tr><td style="border: 1px solid black; width:600px; text-align: left;"><font size=4 color=red><b>실습문제</b></font><br><br>
        <font size=4>
○ 이진 이미지를 출력하는 웹 카메라 만들기<br>

   - threshold() 함수 적용<br>

   - OTSU 이진화 알고리즘 적용<br>
   
   - 적응형 thresholding 적용</font></td></tr>       
    <tr><td style="border: 1px solid black; text-align: left;">
        <img src="https://arome1004.cafe24.com/images/opencv/lecture_image/04_flask06.png" width=30%><img src="https://arome1004.cafe24.com/images/opencv/lecture_image/04_flask07.png" width=30%></td></tr>   
</table>

<table border=1 width=100%>
    <tr><td style="border: 1px solid black; width:600px; height:40px; text-align: center;"><font size=4 color=blue><b>[4차시] 학습요약</b></font></td></tr>       
    <tr><td style="border: 1px solid black; text-align: left;"><font size=3>
        
○ <font color=red>ret, buffer = cv2.imencode()</font> : 영상 인코딩 함수<br>

○ yield() : 반복 실행 중에 중간 과정을 반환할 때 사용<br>

○ FastAPI : 파이썬으로 제작된 웹 프레임워크 (DJango의 축소 버전)<br>
        
○ <font color=red>app = FastAPI()</font> : FastAPI 객체를 app 변수에 할당<br>

○ <font color=red>nest_asyncio.apply()</font> : FastAPI와 uvicorn 연결<br>

○ <font color=red>@app.get("/")</font> : FastAPI에게 어떤 URL이 get방식으로 해당 함수를 실행하는지 알려줌<br>

○ <font color=red>@app.get("/hello")</font> : 서버 주소에 추가적인 URL을 설정<br>

○ <font color=red>@app.get("/hello/{value}")</font> : URL에 value 값을 설정<br>

○ <font color=red>uvicorn.run(app, host="127.0.0.1", port=9600)</font> : 서버 IP, 포트 등을 설정하고 서버를 실행<br>

○ <font color=red>HTMLResponse()</font> : html 문서를 반환하기<br>

○ <font color=red>FileResponse()</font> : 파일로 된 문서를 반환하기<br>

○ <font color=red>TemplateResponse()</font> : 템플릿 형태로 된 문서를 반환하기<br>

○ <font color=red>StreamingResponse(stream_with_context())</font> : 데이터 스트리밍을 반환하기
        
</font></td></tr>   
</table>